In [ ]:
import cadquery as cq
import numpy as np
from jupyter_cadquery.cadquery import show
from jupyter_cadquery import set_defaults, set_sidecar
from jupyter_cadquery.cadquery import PartGroup, Part, Edges, Faces, Vertices
from jupyter_cadquery.cad_animation import Animation


set_defaults(display='sidecar', axes=False, axes0=True, grid=False,
             default_color='lightgrey', tools=True, ortho=True, black_edges=True)

from cq_gears import (SpurGear, HerringboneGear, RackGear, HerringboneRackGear,
                      PlanetaryGearset, HerringbonePlanetaryGearset,
                      BevelGearPair)

set_sidecar('Gears', init=True)

In [ ]:
L = lambda *args: cq.Location(cq.Vector(*args))
Lrz = lambda loc, r: cq.Location(cq.Vector(*loc), cq.Vector(0.0, 0.0, 1.0), r)
C = lambda *args: cq.Color(*args)

In [ ]:
def animate_pinion_and_gear(pinion, gear):
    ratio = pinion.z / gear.z

    asm = (cq.Assembly(name='gears')
           .add(pinion.build(), name='pinion', color=C('goldenrod'),
                loc=L(0.0, 0.0, 0.0))
           .add(gear.build(), name='gear', color=C('lightsteelblue'),
                loc=L(gear.r0 + pinion.r0, 0.0, 0.0)))

    view = show(asm)

    n_frames = 800

    times = np.linspace(0.0, 10.0, n_frames)
    pinion_angles = np.linspace(0.0, 360.0, n_frames)
    gear_angles = -pinion_angles * ratio

    anim = Animation(view.root_group)

    anim.add_track(f'gears/pinion', 'rz', times, pinion_angles)
    anim.add_track(f'gears/gear', 'rz', times, gear_angles)

    return anim.animate(autoplay=True)

In [ ]:
pinion = SpurGear(2.0, 17, 10.0, helix_angle=0.0, bore_d=10.0, chamfer=0.5)
gear = SpurGear(2.0, 29, 10.0, helix_angle=0.0, bore_d=10.0, chamfer=0.5,
                n_spokes=4, spokes_id=20.0, spokes_od=44.0, spoke_width=6.0,
                spoke_fillet=4.0)

animate_pinion_and_gear(pinion, gear)

In [ ]:
pinion = SpurGear(2.0, 17, 10.0, helix_angle=30.0, bore_d=10.0, chamfer=0.5)
gear = SpurGear(2.0, 29, 10.0, helix_angle=-30.0, bore_d=10.0, chamfer=0.5,
                n_spokes=4, spokes_id=20.0, spokes_od=44.0, spoke_width=6.0,
                spoke_fillet=4.0)

animate_pinion_and_gear(pinion, gear)

In [ ]:
pinion = HerringboneGear(2.0, 17, 10.0, helix_angle=-30.0, bore_d=10.0)
gear = HerringboneGear(2.0, 29, 10.0, helix_angle=30.0, bore_d=10.0,
                n_spokes=4, spokes_id=20.0, spokes_od=44.0, spoke_width=6.0,
                spoke_fillet=4.0)

animate_pinion_and_gear(pinion, gear)

In [ ]:
def animate_rack_and_pinion(rack, pinion):
    cfr = pinion.r0 * 2.0 * np.pi
    rp = cfr / pinion.z
    start_x = rp * 1.0
    end_x = rack.length - rp * 2.0
    start_angle = 360.0 / pinion.z * 2.0
    end_angle = (end_x - start_x) / cfr * 360.0
    
    asm = (cq.Assembly(name='gears')
           .add(rack.build(), name='rack', color=C('goldenrod'),
                loc=L(0.0, 0.0, 0.0))
           .add(pinion.build(), name='pinion', color=C('lightsteelblue'),
                loc=Lrz((start_x, pinion.r0, 0.0), start_angle))
          )

    view = show(asm)

    n_frames = 800

    times = np.linspace(0.0, 10.0, n_frames)
    pinion_angles = -np.linspace(0.0, end_angle, n_frames // 2)
    pinion_angles = np.concatenate((pinion_angles,
                                -np.linspace(end_angle, 0.0, n_frames // 2)))
    pinion_tx = np.linspace(start_x, end_x, n_frames // 2)
    pinion_tx = np.concatenate((pinion_tx, 
                                np.linspace(end_x, start_x, n_frames // 2)))

    anim = Animation(view.root_group)

    anim.add_track(f'gears/pinion', 'rz', times, pinion_angles)
    anim.add_track(f'gears/pinion', 'tx', times, pinion_tx)

    return anim.animate(autoplay=True)


In [ ]:
pinion = SpurGear(2.0, 18, 10.0, bore_d=10.0, chamfer=0.5)
rack = RackGear(2.0, length=140.0, width=10.0, height=20.0)

animate_rack_and_pinion(rack, pinion)

In [ ]:
pinion = SpurGear(2.0, 18, 10.0, helix_angle=30.0, bore_d=10.0, chamfer=0.5)
rack = RackGear(2.0, length=140.0, width=10.0, height=20.0, helix_angle=-30.0)

animate_rack_and_pinion(rack, pinion)

In [ ]:
pinion = HerringboneGear(2.0, 18, 10.0, helix_angle=30.0, bore_d=10.0, chamfer=0.5)
rack = HerringboneRackGear(2.0, length=140.0, width=10.0, height=20.0, helix_angle=-30.0)

animate_rack_and_pinion(rack, pinion)

In [ ]:
def animate_planetary_gearset(gearset):
    planet_ratio = gearset.sun.z / (gearset.planet.z * 2.0)
    carrier_ratio = 1.0 / (gearset.ring.z / gearset.sun.z + 1.0)

    asm = gearset.assemble()
    view = show(asm)

    n_frames = 4000

    times = np.linspace(0.0, 60.0, n_frames)
    sun_angles = np.linspace(0.0, 360.0 * 4.0, n_frames)
    carrier_angles = sun_angles * carrier_ratio
    planet_angles = -sun_angles * planet_ratio

    anim = Animation(view.root_group)

    anim.add_track(f'planetary/sun', 'rz', times, sun_angles)
    anim.add_track(f'planetary/planets', 'rz', times, carrier_angles)

    for i in range(gearset.n_planets):
        anim.add_track(f'planetary/planets/planet_{i:02}', 'rz', times,
                       planet_angles - carrier_angles)

    return anim.animate(autoplay=True)

In [ ]:
gearset = PlanetaryGearset(module=1.0,
                           sun_teeth_number=40, planet_teeth_number=20,
                           width=10.0, rim_width=5.0, n_planets=6,
                           bore_d=6.0, chamfer=0.5,
                           sun_build_args={'bore_d': 20.0})

animate_planetary_gearset(gearset)

In [ ]:
gearset = PlanetaryGearset(module=1.0, helix_angle=30.0,
                           sun_teeth_number=40, planet_teeth_number=20,
                           width=10.0, rim_width=5.0, n_planets=6,
                           bore_d=6.0, chamfer=0.5,
                           sun_build_args={'bore_d': 20.0})

animate_planetary_gearset(gearset)

In [ ]:
gearset = HerringbonePlanetaryGearset(
                           module=1.0, helix_angle=30.0,
                           sun_teeth_number=40, planet_teeth_number=20,
                           width=10.0, rim_width=5.0, n_planets=6,
                           bore_d=6.0, chamfer=0.5,
                           sun_build_args={'bore_d': 20.0})

animate_planetary_gearset(gearset)

In [ ]:
def animate_bevel_gear_pair(gearset):
    asm = gearset.assemble(transform_pinion=True)

    view = show(asm)

    n_frames = 4000
    ratio = gearset.pinion.z / gearset.gear.z

    times = np.linspace(0.0, 60.0, n_frames)
    pinion_angles = np.linspace(0.0, 360.0 * 4.0, n_frames)
    gear_angles = -pinion_angles * ratio

    anim = Animation(view.root_group)

    anim.add_track(f'bevel_pair/pinion', 'rz', times, pinion_angles)
    anim.add_track(f'bevel_pair/gear', 'rz', times, gear_angles)

    return anim.animate(autoplay=True)

In [ ]:
gearset = BevelGearPair(module=2.0, gear_teeth=23, pinion_teeth=14,
                        face_width=12.0, axis_angle=90.0, bore_d=12.0,
                        pinion_build_args={'bore_d': 8.0})

animate_bevel_gear_pair(gearset)

In [ ]:
gearset = BevelGearPair(module=2.0, gear_teeth=23, pinion_teeth=14,
                        face_width=12.0, helix_angle=20.0, axis_angle=90.0,
                        bore_d=12.0, pinion_build_args={'bore_d': 8.0})

animate_bevel_gear_pair(gearset)